#Disclaimer to the user
 the prompt travel generation doesn't work in certain colab server locations, so please disconnect, and reconnect the runtime to change the runtime location.

 you can check the server location using the command: !curl ipinfo.io

 Please check locations in the below link https://developers.generativeai.google/available_regions

In [ ]:
# original author: tech-practice ;  original github repo: https://github.com/s9roll7/animatediff-cli-prompt-travel

In [ ]:
# Use this to check the location of the server
!curl ipinfo.io

{
  "ip": "34.16.176.3",
  "hostname": "3.176.16.34.bc.googleusercontent.com",
  "city": "Las Vegas",
  "region": "Nevada",
  "country": "US",
  "loc": "36.1750,-115.1372",
  "org": "AS396982 Google LLC",
  "postal": "89111",
  "timezone": "America/Los_Angeles",
  "readme": "https://ipinfo.io/missingauth"
}

In [ ]:
!pip install flask
!pip install flask-ngrok
!pip install ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 12.0 MB/s eta 0:00:00


In [ ]:
# it will ask you for permission to access the google drive folder,
# please make sure that you are accessing the drive through the same account that you are running the colab notebook with.

!apt-get -qq install aria2

#BOC 11/14
!pip install google-generativeai
import google.generativeai as palm
#Replace the API_key value with your api key generated in PALM site
palm.configure(api_key='')
#EOC 11/14

from google.colab import drive
import os, shutil, gc, time
import gdown

base_path = "/content/animatediff-cli-prompt-travel"
pwd = %pwd
print(pwd)
if pwd != base_path:
  !git clone https://github.com/s9roll7/animatediff-cli-prompt-travel
  %cd {base_path}

  # Mount google drive
  drive.mount("GoogleDrive")
  gd_path = "GoogleDrive/MyDrive"
  gd_output_dir = gd_path + "/AnimateDiffPromptTravel"+"/output"

  # create a folder called "AnimateDiffPromptTravel" under "GoogleDrive/MyDrive"
  os.makedirs(gd_output_dir, exist_ok=True)

  #Install all dependencies under the base path
  !pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
  !pip install .[dev]
  !pip install xformers mediapipe
  !pip install onnxruntime-gpu pandas

Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 120882 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.2_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.2) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.36.0-1_amd64.deb ...
Unpacking aria2 (1.36.0-1) ...
Setting up libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.2) ...
Setting up libaria2-0:amd64 (1.36.0-1) ...
Setting up aria2 (1.36.0-1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.3 MB/s eta 0:00:00


In [ ]:
# AnimateDiff MotionModule download, you can customize this in the last code cell
def dnld_mm(mm):
  data_dir = "data"
  motion_module = mm
  motion_module_path = f"models/motion-module/{motion_module}.ckpt"
  motion_module_file_path = os.path.join(data_dir, motion_module_path)
  if not os.path.exists(motion_module_file_path):
    url = f"https://huggingface.co/guoyww/animatediff/resolve/main/{motion_module}.ckpt"
    !aria2c {url} -o {motion_module_file_path} #Download the MM model checkpoint file from the URL
  return motion_module_path

In [ ]:
# Standard Diffussion model download
# feel free to experiment other models, you can customize this in the last code cell
def dnld_sd(url, name):
  data_dir = "data"
  model_url = url
  model_name = name
  #model_path = f"models/sd/{model_name}.safetensors"
  model_path = f"models/sd/{model_name}"
  model_file_path = os.path.join(data_dir, model_path)
  if not os.path.exists(model_file_path):
    !aria2c {model_url} -o {model_file_path} #Download the MM model checkpoint file from the URL
  return model_path


In [ ]:
# Colab free version has limited RAM. SO, delete the images from controlnet folder so that the program runs without interruptions.
def cleanup(folder_path):
  if len(os.listdir(folder_path)) > 1:
    os.remove(f"{folder_path}/0000.png")
    os.remove(f"{folder_path}/0016.png")
    os.remove(f"{folder_path}/0032.png")

In [ ]:
# # take about 14-15 minutes depending on the diffussion model that you chose, and the total number of frames that are geenrated.
def inference(config, W, H, L, C):
  !animatediff generate -c {config} -W {W} -H {H} -L {L} -C {C}


In [ ]:
# # Copy generated contents to Google drive
def cpy_to_drive():
  output_dir = "output"
  gc.collect()

  def get_new_dir(parent_dir):
    sub_dirs = []
    for path in os.listdir(parent_dir):
      dir_path = os.path.join(parent_dir, path)
      print("dir_path = ",dir_path)
      if os.path.isdir(dir_path):
        sub_dirs.append(dir_path)
    print("sub_Dirs value =",sub_dirs,"value of key =",os.path.getmtime)
    return max(sub_dirs, key = os.path.getmtime)

  result_dir = get_new_dir(output_dir)
  print("Result_dir value = ",result_dir)
  print("gd_output_dir = ",gd_output_dir)
  dest_dir = os.path.join(gd_output_dir, os.path.basename(result_dir))
  print("Destination Directory =",dest_dir)
  shutil.copytree(result_dir, dest_dir, dirs_exist_ok = True) #Copy the file directory tree to the google drive folder
  print(f"🎉 The output files have been copied into {dest_dir}. (in your connected Google Drive) See you there! 🎉")
  return dest_dir

**FLASK CONFIGURATION**

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
import pickle
import numpy as np

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!cd /content/animatediff-cli-prompt-travel/GoogleDrive/MyDrive/AnimateDiffPromptTravel/UI
!chmod +x ngrok
# Replace  your_authtoken wih your generated Authentication Token in  with the authtoken Generated in the ngrok site
!./ngrok authtoken your_authtoken



--2023-12-04 01:57:47--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  21.4MB/s    in 0.6s    

2023-12-04 01:57:48 (21.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!cat /root/.ngrok2/ngrok.yml

authtoken: 2Z17WkUquPo3DEz99dXpB4ABrva_3occ1kz195SBTfhfVMfP7


In [ ]:
!pip install --upgrade pydrive

In [ ]:
# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

def videoRendering(video_name):
    print("Video Rendering is Starting")
    print("video_name ===",video_name)
    file_list = drive.ListFile({'q': f"title='{video_name}' and trashed=false"}).GetList()
    print("file_list=",file_list)
    shareable_link = ""
    for file in file_list:
        print(file['title'], file['id'])
        if file['id']:
            # Build the shareable link
            shareable_link = f"https://drive.google.com/file/d/{file['id']}/view?usp=sharing"
            print("shareable_link = ", shareable_link)
        else:
            shareable_link = ""

    return shareable_link
    # return render_template('T2V_frontend.html', video_url = "https://drive.google.com/file/d/14SdcrT58QPQkujLpjQDaIaQn4zXCTU2b/view?usp=sharing")


In [ ]:
cd /content/animatediff-cli-prompt-travel/GoogleDrive/MyDrive/AnimateDiffPromptTravel/

/content/animatediff-cli-prompt-travel/GoogleDrive/MyDrive/AnimateDiffPromptTravel


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

from pathlib import Path
#create Flask app
app = Flask(__name__, static_url_path='/static')

#run FLask app with ngrock
run_with_ngrok(app)

video_file_name = ""

@app.route("/", methods = ['GET', 'POST'])
def home():
    if request.method == 'POST':
        # Retrieve values from the form
        gen_prompt_val= request.form.get('generatePromptCheckbox')

        if gen_prompt_val == 'on':
          generate_prompt = True
        else:
          generate_prompt = False


        fps_val = request.form.get('fpsInput')
        duration = request.form.get('durationSelect')
        file_format = request.form.get('fileFormatSelect')
        head_prompt_val = request.form.get('headPrompt')
        prompt_travel_val = request.form.get('promptTravel')
        tail_prompt_val = request.form.get('tailPrompt')
        negative_prompt_val = request.form.get('negativePrompt')
        motion_module = request.form.get('motionModuleSelect')
        stable_diffusion_url = request.form.get('stableDiffusionModelURL')
        sd_model_name = request.form.get('nameOfSDModel')
        config_prompt = request.form.get('configPrompt')
        w_value = request.form.get('wValue')
        h_value = request.form.get('hValue')
        l_value = request.form.get('lValue')
        c_value = request.form.get('cValue')

        print("motion_module value =",motion_module,
              "fps_val =",fps_val,"l_value = ",l_value,"c_value = ",c_value)

        def json_modif(config, L, mmpath, sdpath):
            #------------------------------------------------
            models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]

            model = models[0].name

            # no_lines = 16 #@param {allow-input: true, type: "string"}
            no_lines = duration
            # fps = 8 #@param {allow-input: true, type: "integer"}

            fps = fps_val

            print("FPS Value=",fps,"-- L Value=",L)


            op_format = file_format
            head_prompt = head_prompt_val
            prompt_travel = prompt_travel_val
            tail_prompt = tail_prompt_val
            # n_prompt = negative_prompt_val
            n_prompt = negative_prompt_val.split(",")

            prompt = f"""
            You are an expert at text genetation.

            expand the prompt into {no_lines} lines story should have the coherence from one line to the next line like a video

            Prompt: {prompt_travel}

            """
            #Generation of the prompt travel
            completion = palm.generate_text(
              model=model,
              prompt=prompt,
              temperature=0.1,
              # The maximum length of the response
              max_output_tokens=800,
            )


            #split the prompt travel into separate sentences

            prompt_map = completion.result.split(".")

            prompt_map1 = []
            for i in range(len(prompt_map)):
              if i%2 != 0:
                prompt_map1.append(prompt_map[i])



            #overwrite the existing data in the json file
            import json
            # Read the existing JSON file
            with open(config, 'r') as json_file:
                data = json.load(json_file)
            i = 0

            #path to the sd model
            data["path"] = sdpath
            #path to the mm model
            data["motion_module"] = mmpath

            #head prompt
            data['head_prompt'] = head_prompt

            #prompt travel
            for line in prompt_map1:
              pos = str(i)
              data['prompt_map'][pos] = line
              # i+=fps
              i+=int(fps)

            #tail prompt
            data['tail_prompt'] = tail_prompt

            #negative prompt
            data['n_prompt'] = n_prompt

            #-----------------------------------------------
            #Output details
            data['output']['format'] = op_format
            data['output']['fps'] = fps

            # Write the updated data back to the JSON file
            with open(config, 'w') as json_file:
                json.dump(data, json_file, indent=2)

            return config, prompt_map1
            #------------------------------------------------

        display_prompt = True #@param {type:"boolean"}

        # -- Parameters for the program to run --

        # Please make sure that you have inputs for the prompt to travel throughout the frames.
        # if you want to generate video with 128 frames with 8FPS, then write the prompt accordingly.
        # The guideline for the prompt can be seen directly in the original repo

        # Motion Module 
        mm = motion_module

        # Standard Diffussion model
        url = stable_diffusion_url
        name = sd_model_name

        %cd {base_path}
        config = Path(config_prompt)
        print("value of config:",config,
              "type of config:",type(config))

        # Width of the frame
        W = w_value
        # Height of the frame
        H = h_value
        # Total number of frames
        L = l_value

        C = c_value

        print("mm value:",mm)
        print("url name:",url)
        print("sd module name:",name)
        print("W:",W,"  H:",H," L:",L," C:",C)

        mmpath = dnld_mm(mm)
        sdpath = dnld_sd(url, name)

        if generate_prompt:
            # editing the config file (prompt file)
           config, prompt_map1 = json_modif(config, L)

           # Check the value of the checkbox
           if display_prompt:
              # from ipywidgets import Output
              from IPython.display import clear_output
              clear_output()
              print(prompt_map1)
              # Add actions to perform when the checkbox is checked
           else:
               print("Checkbox is unchecked.")
               # Add actions to perform when the checkbox is unchecked


        # Clean up of the control net reference image folder
        # folder_path = "data/controlnet_image/test/controlnet_softedge"
        folder_path = "/content/animatediff-cli-prompt-travel/data/controlnet_image/test/controlnet_softedge"
        cleanup(folder_path)
        # folder_path = "data/controlnet_image/test/controlnet_openpose"
        folder_path = "/content/animatediff-cli-prompt-travel/data/controlnet_image/test/controlnet_openpose"
        cleanup(folder_path)


        inference(config, W, H, L, C)
        dst_dr = cpy_to_drive()
        print("dst_dr value =",dst_dr)

    #return render_template('GoogleDrive/My Drive/AnimateDiffPromptTravel/UI/T2V_frontend.html')
    return render_template('T2V_frontend.html', video_url = None)

if __name__ == '__main__':
    app.run()



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://91ea-34-16-176-3.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 02:00:20] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 02:00:20] "GET /static/script.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 02:00:20] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 02:00:21] "GET /favicon.ico HTTP/1.1" 404 -
